In [1]:
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

import pandas as pd

pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
import seaborn as sns

from scipy.stats import distributions

import pickle

# add utilities directory to path
import os, sys
util_path = os.path.abspath('utilities_and_data')
if util_path not in sys.path and os.path.exists(util_path):
    sys.path.insert(0, util_path)

# import from utilities
import stan_utility
import stan_helper

import pystan
from IPython.display import Image

In [2]:
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 22}
import matplotlib
matplotlib.rc('font', **font)


In [3]:
from psis import psisloo

In [4]:
def get_stan_model(model_name, model_code, get_new=False):
    path = os.path.abspath(os.path.join(os.path.curdir, model_name))

    if not get_new and os.path.exists(path):
        print("Model exists already! Returning pickle.")
        return pickle.load(open(path, 'rb'))
    
    print("Path doesn't exist. Compiling model. It might take few minutes...")
    import pystan
    sm = pystan.StanModel(model_code=model_code)
    with open(model_name, 'wb') as f:
        pickle.dump(sm, f)
    return sm


Load model

In [5]:
d = np.loadtxt("data/factory.txt")

## (3) hierarchical model

Hierarchical modeling is a statistically rigorous way to make scientific inferences about a population based on many groups/observations. Humanly speaking, it's a middle ground between pooling and separate modeling. We will have the same formula as separate:

$$ y_{i,m} = \alpha_m + \beta_m \times \text{machine}_{i,m} + \epsilon_m $$

But $\alpha$ and $\beta$ come from a common group distribution:

$$\alpha_{c} \sim \mathcal{N}(\mu_{\alpha}, \sigma_{\alpha}^2)$$
$$\beta_{c} \sim \mathcal{N}(\mu_{\beta}, \sigma_{\beta}^2)$$

So we model coefficients and predictions. That's why it's called hierarchical/multilevel modeling.

In [8]:
x = np.tile(np.arange(1, 7), d.shape[0])
y = d.ravel();y
N = len(x)
data = dict(
    N = N,
    K = 7,  # 6 machines
    x = x,  # group indicators
    y = y,   # observations
    target_machine = 6  # 6th machine
)
print(x.shape,y.shape,N)

(30,) (30,) 30


Our linear model:
$$ y = \beta_0 + \beta_1 \times x + e $$

$e$ is an error term sampled from $N(\mu, \sigma)$

Priors for each machine is not fixed, but rather depend on other latent variables.

In [9]:
#  Comparison of k groups with common variance and
#  hierarchical prior for the mean

data_code = '''
data {
    int<lower=0> N; // number of data points
    int<lower=0> K; // number of groups
    int<lower=1,upper=K> x[N]; // group indicator
    vector[N] y; //
    int<lower=0> target_machine;
}
parameters {
    real<lower=70, upper=110> mu0; // uniform prior mean
    real<lower=0, upper=20> sigma0; // uniform prior std
    vector[K] mu;         // group means
    real<lower=0> sigma;  // common std
}
model {
  mu ~ normal(mu0, sigma0); // population prior with unknown parameters
  y ~ normal(mu[x], sigma);
  //for (i in 1:N)
  //mu7 ~ normal(mu0, sigma0);  // Is this correct? Idk. mu7 ~ normal(mu0, sigma0) gave me low mean around 19.
}
generated quantities {
  real ypred;
  real mu7;
  real ypred7;
  vector[6] ypred_all;
  vector[N] log_lik;
  ypred = normal_rng(mu[target_machine], sigma);
  for (i in 1:N)
    log_lik[i] = normal_lpdf(y[i] | mu[x[i]], sigma);
    
  mu7 = normal_rng(mu0,sigma0);
  ypred7 = normal_rng(mu7, sigma);
  
  for (i in 1:6)
    ypred_all[i] = normal_rng(mu[i], sigma);
}
'''

In [10]:
# sm_hierarchical =  get_stan_model("factory_hierarchical_posterior", data_code, get_new=True)
sm_hierarchical =  get_stan_model("factory_hierarchical_posterior", data_code)

Model exists already! Returning pickle.


In [11]:
fit_hierarchical = sm_hierarchical.sampling(data=data, seed=194838, chains=10, iter=4000);fit_hierarchical

Inference for Stan model: anon_model_b90bc0e16312b4e8431f9ae51b62b3a4.
10 chains, each with iter=4000; warmup=2000; thin=1; 
post-warmup draws per chain=2000, total post-warmup draws=20000.

               mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
mu0           92.87    0.05   5.79  80.88  89.25  92.96  96.62 104.22  15512    1.0
sigma0        12.19    0.08    4.1   3.94   9.21  12.16  15.37  19.38   2446    1.0
mu[0]         80.85     0.1   6.75  68.03   76.2  80.81  85.33  94.72   4718    1.0
mu[1]        102.41    0.06   6.37  89.93  98.18 102.48 106.66 114.92  10669    1.0
mu[2]         89.22    0.04   6.05   77.4  85.24  89.22  93.29 101.07  20000    1.0
mu[3]        106.22    0.09   6.82   92.8 101.66 106.33 110.87 119.13   5593    1.0
mu[4]         90.79    0.04   6.04   78.8  86.86  90.77  94.78 102.67  20000    1.0
mu[5]         88.01    0.06   6.19  75.75   83.9  88.08  92.16 100.04  10732    1.0
mu[6]         92.78    0.11  14.02  64.32   84.5   92

In [12]:
# Extract samples in different forms for different parameters.
# permuted : bool
#    If True, returned samples are permuted. All chains are
#    merged and warmup samples are discarded.
samples_hierarchical = fit_hierarchical.extract(permuted=True)

# stan_utility.check_treedepth(fit_hierarchical)
# stan_utility.check_energy(fit_hierarchical)
# stan_utility.check_div(fit_hierarchical)

In [27]:
log_likelihood_hierarchical = samples_hierarchical['log_lik'];

*log_likelihood* is an S-by-n matrix, where S is the number of posterior draws and n = 30 is the total number of observations

## Likelihood and posterior

|Quality | Likelihood | Posterior |
|-|:-:|:-:|
|Good|||
|Bad| | |

![simulation_methods.png](images/simulation_methods.png)

## For each of the six machines, compute and report the expected utility of the products of that machine.

In [27]:
# Expected Utility for Machine 1
def get_expected_utility(machine_num):
    utility_sum = 0
    S = len(samples_hierarchical['ypred_all'][machine_num])
    for i in range(S):
        if samples_hierarchical['ypred_all'][machine_num][i] < 85:
            utility_sum -= 100
        else :
            utility_sum += 100
    return utility_sum / S

expected_utilities = [get_expected_utility(i) for i in range(6)]; 
print("==Expected utility of the products of six machinee.==")
for i in range(len(expected_utilities)):
    print("{}th machine's Expected utility of the products: ${:.2f}".format(i+1, expected_utilities[i]))

==Expected utility of the products of six machinee.==
1th machine's Expected utility of the products: $33.33
2th machine's Expected utility of the products: $0.00
3th machine's Expected utility of the products: $33.33
4th machine's Expected utility of the products: $33.33
5th machine's Expected utility of the products: $100.00
6th machine's Expected utility of the products: $33.33


## Rank the machines based on the expected utilities and explain briefly what these values tell about the quality of these machines. E.g. Tell which machines are profitable and which are not (if any).

|Rank|Machine|Expected utility of products|
|:-:|:-:|-:|
|1|5|\$100.00|
|2|1|\$33.33|
|3|3|\$33.33|
|4|4|\$33.33|
|5|5|\$33.33|
|6|2|\$0.00|


## Compute and report the expected utility of the products of a new (7th) machine.

In [30]:
def get_7th_expected_utility():
    utility_sum = 0
    S = len(samples_hierarchical['ypred7'])
    for i in range(S):
        if samples_hierarchical['ypred7'][i] < 85:
            utility_sum -= 100
        else :
            utility_sum += 100
    return utility_sum / S

expected_utility_7th = get_7th_expected_utility()
print("==Expected utility of the products of 7th machine.==")
print("7th machine's Expected utility of the products: ${:.2f}".format(expected_utility_7th))

==Expected utility of the products of 7th machine.==
7th machine's Expected utility of the products: $29.79


|Machine|Expected utility of products|
|:-:|-:|
|1|\$33.33|
|2|\$0.00|
|3|\$33.33|
|4|\$33.33|
|5|\$100.00|
|6|\$33.33|
|7|\$29.79|


## Based on your analysis, discuss briefly whether the company owner should buy a new (7th) machine.

The expected utility of products is $29.79 which generates profit. Thus the owner should buy a new one. 

# Ignore below

|Decision d | good quality | bad quality |
|-|:-:|:-:|
|Not Buy|0|0|
|Buy| +100| -100|

## Utility matrix U(x)

|Action d | Conditioinal utility |
|-|:-:|:-:|
|Not Buy|0|
|Buy| |